# Biodiversity Intactness Index change in Phoenix, AZ
### Author: Bailey Jørgensen
**Repository:** https://github.com/jorb1/eds220-BII

## About:

**Purpose:**
In 2021, Maricopa County —home to the Phoenix metropolitan area— was identified as the U.S. county with the most significant increase in developed land since 2001. This rapid urban sprawl has profound implications for biodiversity and the health of surrounding natural ecosystems.

In this notebook, I will investigate the impacts of urban expansion by analyzing a dataset that captures values for the Biodiversity Intactness Index (BII). Apecifically, I will examine changes in BII in the Phoenix county subdivision area between 2017 and 2020, shedding light on how urban growth affects biodiversity over time.

**Highlights:**
1. 

2. 

3. 

4. 

**About the data:**
1. The first data set is is the Biodiversity Intactness Index (BII) Time Series. Access the io-biodiversity collection from the Microsoft Planetary Computer STAC catalog. I will be using the 2017 and 2020 rasters covering the Phoenix subdivision. 

2. The second data set is the Phoenix Subdivision Shapefile Download the Phoenix subdivision polygon from the Census County Subdivision shapefiles for Arizona. All legal boundaries and names are as of January 1, 2024. The 2024 TIGER/Line Shapefiles were released on September 25, 2024. https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2022&layergroup=County+Subdivisions

Both of these datasets were accessed for this analysis on 12/2/2024.

In [12]:
# Load Libraries
import pandas as pd
import geopandas as gpd
import planetary_computer
import pystac_client
import rich.table
from geogif import gif
import numpy as np
import matplotlib.pyplot as plt
import rioxarray as rioxr
from IPython.display import Image 
from shapely.geometry import box
import xarray as xr
import os
import rasterio
from rasterio.windows import from_bounds

In [2]:
# Read in shapefile data for Arizona
arizona = gpd.read_file('data/tl_2022_04_cousub.shp')

In [3]:
# Access the MPC catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

catalog.get_collections()
collections = list(catalog.get_collections())

# Print the number of collections
print('Number of collections:', len(collections))

#Pull out the Impact Observatory collection
io_collection = catalog.get_child('io-biodiversity')
io_collection

Number of collections: 124


<CollectionClient id=io-biodiversity>

Now that we have our data, we need to specify the temporal and spatial information we are interested in. I will create a bounding box, that specifies our spatial area of interest.

As we can see from the description of the Impact Observatory Collection above, the only dates represented in the data are our time range of interest (2017-2020). As such, there is no need to specify a temporal variable. 

In [4]:
#time_range = "2017-01-01/2020-01-01"

bbox_of_interest = [-112.826843, 32.974108, -111.184387, 33.863574]
#search = catalog.search(collections=["io-biodiversity"], bbox=bbox_of_interest)

#items = list(search.items())
#for item in items:
#    print(item)

In [5]:
# Catalog search
search = catalog.search(
    collections = ['io-biodiversity'],
    bbox = bbox_of_interest)

# Get items from search
items = search.item_collection()

# Determine number of items in search
print(f'There are {len(items)} items in the search.')

There are 4 items in the search.


In [6]:
# Get first item in the catalog search
item = items[0]
type(item)

pystac.item.Item

Our search returned four STAC Items. We can tell from their IDs that that they contain data for the same area but for different times, specifically the years 2017 through 2020. Let's display the available assets and properties for the 2017 Item.

In [7]:
asset_table = rich.table.Table("Asset Key", "Asset Title")
for key, value in items[-1].assets.items():
    asset_table.add_row(key, value.title)
asset_table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Asset Title                     ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ data             │ Biodiversity Intactness         │
│ tilejson         │ TileJSON with default rendering │
│ rendered_preview │ Rendered preview                │
└──────────────────┴─────────────────────────────────┘

In [8]:
property_table = rich.table.Table("Property Name", "Property Value")
for key, value in sorted(items[-1].properties.items()):
    property_table.add_row(key, str(value))
property_table

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property Name  ┃ Property Value                                                                                 ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ datetime       │ None                                                                                           │
│ end_datetime   │ 2017-12-31T23:59:59Z                                                                           │
│ proj:epsg      │ 4326                                                                                           │
│ proj:shape     │ [7992, 7992]                                                                                   │
│ proj:transform │ [0.0008983152841195215, 0.0, -115.38597824385106, 0.0, -0.0008983152841195215,                 │
│                │ 34.74464974521749, 0.0, 0.0, 1.0]                                                              │
│ start_datetime │ 2017-01-01T00:00:00Z                                                                           │
└────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

#### Let's look at a single raster. 

 We want to open `item.assets['data'].href` to get the xarray. 

In [ ]:
bio_single = rioxr.open_rasterio(item.assets['data'].href)
bio_single

This rater is way bigger than our area of interest. To verify this and then clip the raster, let’s make a gpd.GeoDataFrame from the bbox coordinates:

In [ ]:
# Bounding box as geodataframe
box_df = gpd.GeoDataFrame(geometry=[box(*bbox_of_interest)],
                 crs='epsg:4326') 

In [ ]:
# Clip raster to bounding box, ensuring the crs' are the same
bio_single = bio_single.rio.clip_box(*box_df.to_crs(bio_single.rio.crs).total_bounds)

In [ ]:
# Check raster dimensions
bio_single.dims

# Check the shape of the raster
bio_single.shape

read the data assets for the four Items into a single xarray.DataArray?

In [ ]:
rasters = []
for item in items: 
    # Access the image asset
    bio_single = rioxr.open_rasterio(item.assets['data'].href) 
    # Clip the raster
    bio_single = bio_single.rio.clip_box(*box_df.to_crs(bio_single.rio.crs).total_bounds)
    # Select the first three bands
    bio_single = bio_single.sel(band = [1, 2, 3])
    
    rasters.append(bio_single)

In [ ]:
data_array = bio_single.squeeze().compute()

In [ ]:
grid = data_array.plot(col="time", cmap="Greens", robust=True)
grid;